<a href="https://colab.research.google.com/github/meliksahb/Design-of-Intelligent-Machines-ME536-/blob/main/Week10_e2738425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Full Name: Melikşah Beşir
#Student ID: e2738425

# Try to Tell Apples, Oranges and else apart

Images for this assignment are provided at github and they will be automatically downloaded right after imports. When the files are unzip there should be a folder:  ```AppleOrange```.
These are sample images, you can test with more if you want. While testing just for fun, I can try other stuff as well.

Your objective is to convert <font color="magenta"> **Apples to magenta** </font>, <font color="blue"> **Oranges to Blue** </font> , end blur everything else! While changing color, try to keep the original shading (i.e. try not provide a flat single color if possible, keep the shadings to the best you can).  

<font color="red">Do not be **too picky** and lose too much time</font>. Variety of images are provided so that you get the idea that generalizable / perfect filters are not easy to build. Yet your function is expected to work on more than one image at an acceptable level.  

By the same token, you can use the ```fakes```, also for for fun to see how your algorighm works on unrealted images, and why a general filter is not that easy...  

At the end as usual you are expected to **clear all outputs** and then save this file as **Week10_student_id.ipynb** and upload to the assignment at [ODTU Class](https://odtuclass2024f.metu.edu.tr/mod/assign/view.php?id=100364).  


# imports as usual
You are only allowed to use concepts related to what we have seen in class and use only the following imports. You can import sub-libraries with new names etc. but NO NEW LIBRARIES

In [ ]:
# not that all of them are necessary, but you are not allowed to import any new library
# yet as before, you can import sub libraries: i.e.:
#       from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pimg # check this out this is new
from numpy import cos, arccos, sin, pi, round
from numpy.linalg import matrix_rank as rank
from numpy.linalg import svd, eig
from scipy.linalg import orth
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
import cv2 as cv
from sklearn.decomposition import PCA
from skimage.segmentation import slic
from skimage.util import img_as_float
from skimage.color import rgb2lab
from PIL import Image # good old pillow
import sklearn as skl # famous sci-kit learn
import skimage as ski # equally famous sci-kit image
!rm bug_numpy_utils.py 2>/dev/null  # at the first run file does not exits but error should print
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/bug_numpy_utils.py
from bug_numpy_utils import MatPrint, CData, text2mat # note that once these files are downloaded you can read their content.
!rm me536utils.py 2>/dev/null  # at the first run file does not exits but error should print
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/me536utils.py
from me536utils import RotMat
import os

## get images


In [ ]:
!rm AppleOrange.zip 2>/dev/null # just in case
!wget https://github.com/bugrakoku/data4all/raw/main/AppleOrange.zip # get the zip file
!unzip AppleOrange.zip

# Get prepared
Below you can check images, perform tests runs, find critical color values etc.  
Leave all your preparation code so that I can see how you have reached the final implementation of the function. This part I will NOT run for evaluation! I will only run the ```AorO()``` and ```AorO2()``` functions in my tests!  

You can add as many code and text cells as you like below. But at the end, as I said above, I will just call the ```AorO()``` or ```AorO2()``` function.  
And please **DO CLEAR ALL OUTPUTS**!

Recall that you are not restricted to <font color="red">R</font>
<font color="green">G</font>
<font color="blue">B</font> at all!

# Part I: Manual threshold determination.
You can determine as many thresholds as you like.  


### Add explanations of what you do and why regarding the code(s) below

## ```AorO()``` function

In [ ]:
'''
You are to properly implement the following function so that
Apples become magenta, orange blue in color, and rest is blurred where blurring is visually detectable.

'''

def AorO(img):
    '''
    this function takes the path of an image, loads the image
    converts the color of apples to magenta, color of oranges to blue and blurs the rest
    you are to use some hand crafted thresholds here
    returns the modified image
    '''

    # following is just a stub, you can totally get rid of it!
    oi = pimg.imread(img) # read as RGB image
    # Convert to HSV
    hsv = cv.cvtColor(oi, cv.COLOR_RGB2HSV)
    # hsv is in OpenCV format: H in [0,180), S,V in [0,255]
    # reference: https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
    # Determining color spaces reference: https://www.hbmacit.com/2020/09/15/python-ve-opencv-ile-rgb-uzerinde-hsv-renk-kodu-tespiti/
    # Red apples has two mask because H value is at the beginning of the angles:
    # I asked GPT how to detect red/green/orange in image in hsv format
    mask_red_lower = cv.inRange(hsv, (0,50,50), (10,255,255))
    mask_red_upper = cv.inRange(hsv, (150,50,50), (180,255,255))
    mask_red = cv.bitwise_or(mask_red_lower, mask_red_upper)

    # Green apples: hue ~ [35,85]
    mask_green = cv.inRange(hsv, (20,50,50), (85,255,255))

    # Combine red and green apple masks
    mask_apple = cv.bitwise_or(mask_red, mask_green)

    # Orange: hue ~ [10,30]
    mask_orange = cv.inRange(hsv, (10,50,50), (23,255,255))

    # Combine fruit masks
    mask_fruit = cv.bitwise_or(mask_apple, mask_orange)

    # Blur the entire original image
    blurred = cv.GaussianBlur(oi, (51,51), 0)
    blurred_hsv = cv.cvtColor(blurred, cv.COLOR_RGB2HSV)

    # Apples (red or green) -> magenta hue
    # Magenta ~ 300 degrees => 300/2 = 150 in OpenCV hue
    hue_apple = 150

    # Oranges -> blue
    # Blue ~ 240 degrees => 240/2 = 120 in OpenCV hue
    hue_orange = 120

    # Keep original S and V from oi
    orig_s = hsv[:,:,1]
    orig_v = hsv[:,:,2]

    final_hsv = hsv.copy()

    # For background (not fruit), use blurred HSV
    # mask_fruit = 1 where fruit, 0 where background
    # I asked GPT how to mask background instead of red/green and orange
    background_mask = (mask_fruit == 0)
    final_hsv[background_mask] = blurred_hsv[background_mask]

    # Recolor apples
    apple_locs = (mask_apple > 0)
    final_hsv[apple_locs, 0] = hue_apple
    final_hsv[apple_locs, 1] = orig_s[apple_locs]
    final_hsv[apple_locs, 2] = orig_v[apple_locs]

    # Recolor oranges
    orange_locs = (mask_orange > 0)
    final_hsv[orange_locs, 0] = hue_orange
    final_hsv[orange_locs, 1] = orig_s[orange_locs]
    final_hsv[orange_locs, 2] = orig_v[orange_locs]

    # Convert back to RGB and scale to [0,1]
    # I asked how to convert hsv to RGB
    final_img = cv.cvtColor(final_hsv, cv.COLOR_HSV2RGB).astype(np.float32)/255.0


    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    ax[0].imshow(oi)
    ax[0].set_title("Original")
    ax[0].axis('off')

    ax[1].imshow(final_img)
    ax[1].set_title("Processed")
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

    return final_img

# image_folder = '/content/AppleOrange'

# for filename in os.listdir(image_folder):
#     if filename.endswith(('.jpg', '.png')): # Process only image files
#         image_path = os.path.join(image_folder, filename)
#         processed_image = AorO(image_path)

The function begins by loading the input image in RGB format and converting it to the HSV color space. I did this because HSV separates color information (hue) from intensity and brightness, making it easier and more accurate because hsv creates a sphere in 3D space and represents colors more precisely.

Next, I created masks to identify the regions corresponding to apples and oranges. Apples were split into red and green varieties due to differences in their hue ranges, and the two masks were combined into a single apple mask. Oranges were similarly identified based on their hue values, and all the fruit masks were merged to distinguish fruit areas from the background.

To visually separate the background from the fruits, I applied a Gaussian blur to the entire image. This blurred version is used for areas not covered by the fruit masks, ensuring that the fruits remain sharp and prominent while the background is de-emphasized.

For recoloring, I changed the hue of apples to magenta (hue = 150) and oranges to blue (hue = 120). This transformation preserves the original saturation and brightness of these regions to maintain realistic visual effects. The blurred background is combined with the recolored fruits to produce the final image.

Finally, I converted the modified image back to the RGB format for standard visualization and displayed both the original and processed images side by side. This approach highlights the fruits effectively by altering their colors while softening the background for better visual emphasis.

# Part II: Use some clustering methods to find thresholds automatically
Assuming that there are apples and oranges in the image, analyze the colors in the image (such as histograms, or anything else you find fit), run some clustering algorithms on them and generate the image.

### Add explanations of what you do and why regarding the code(s) below

## ```AorO2()``` function

In [ ]:
'''
You are to properly implement the following function so that
Apples become magenta, orange blue in color, and rest is blurred where blurring is visually detectable.

'''

def AorO2(img):
    '''
    This function takes the path of an image, loads the image,
    converts the color of apples to magenta, color of oranges to blue and blurs the rest
    using clustering thresholds.
    Returns the modified image.
    '''
    # Load the image
    oi = cv.imread(img)  # Reads the image in BGR format

    # Convert the image to HSV color space
    oi_hsv = cv.cvtColor(oi, cv.COLOR_BGR2HSV)

    # Define HSV ranges for red (two ranges due to hue wrap-around), green, and orange
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([165, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    lower_green = np.array([20, 50, 50])
    upper_green = np.array([85, 255, 255])

    lower_orange = np.array([10, 100, 50])
    upper_orange = np.array([23, 255, 255])

    # Create masks for red, green, and orange
    red_mask1 = cv.inRange(oi_hsv, lower_red1, upper_red1)
    red_mask2 = cv.inRange(oi_hsv, lower_red2, upper_red2)
    red_mask = cv.bitwise_or(red_mask1, red_mask2)  # Combine the two red masks

    green_mask = cv.inRange(oi_hsv, lower_green, upper_green)
    orange_mask = cv.inRange(oi_hsv, lower_orange, upper_orange)

    # Combine red and green masks into one apple mask
    apple_mask = cv.bitwise_or(red_mask, green_mask)

    # Initialize the output image
    output_img = np.copy(oi)

    # Apply magenta color to apples
    output_img[apple_mask > 0] = [255, 0, 255]  # Magenta in BGR

    # Apply blue color to oranges
    output_img[orange_mask > 0] = [255, 0, 0]  # Blue in BGR

    # Blur the rest of the image
    blurred_img = cv.GaussianBlur(oi, (51, 51), 0)
    not_fruit_mask = ~(apple_mask > 0) & ~(orange_mask > 0)
    output_img[not_fruit_mask] = blurred_img[not_fruit_mask]

    return output_img

# def plot_processed_images(image_folder, process_function):
#     """
#     Plots the original and processed images side by side.

#     Args:
#         image_folder (str): Path to the folder containing the images.
#         process_function (function): The function used to process the images.
#     """
#     # List all image files in the folder
#     image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

#     # Iterate over each image file
#     for filename in image_files:
#         image_path = os.path.join(image_folder, filename)

#         # Read the original image
#         original_image = cv.imread(image_path)
#         original_image_rgb = cv.cvtColor(original_image, cv.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

#         # Process the image
#         processed_image = process_function(image_path)
#         processed_image_rgb = cv.cvtColor(processed_image, cv.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

#         # Plot original and processed images side by side
#         plt.figure(figsize=(12, 6))
#         plt.subplot(1, 2, 1)
#         plt.imshow(original_image_rgb)
#         plt.title('Original Image')
#         plt.axis('off')

#         plt.subplot(1, 2, 2)
#         plt.imshow(processed_image_rgb)
#         plt.title('Processed Image')
#         plt.axis('off')

#         plt.suptitle(f"Image: {filename}", fontsize=16)
#         plt.show()

# # Example usage:
# image_folder = '/content/AppleOrange'

# # Call the plotting function with the folder path and the processing function
# plot_processed_images(image_folder, AorO2)

I started by loading the image using OpenCV and converting it from BGR to HSV color space because HSV is better suited for identifying specific colors. I defined thresholds for the HSV ranges to detect apples and oranges. I used two ranges for red because red wraps around the hue spectrum, and I included a green range to account for green apples. For oranges, I defined a separate range to detect their color. I then created masks.

I combined the red and green masks to form an "apple mask" because both red and green pixels could represent apples. For oranges, I used the orange mask directly. I applied magenta ([255, 0, 255] in BGR) to the apple mask and blue ([255, 0, 0] in BGR) to the orange mask because these colors visually differentiate the fruits.

To handle the rest of the image, I applied a Gaussian blur with a large kernel size to create a soft, visually detectable blur. I identified the areas not belonging to apples or oranges using logical negation of the masks. I then replaced those areas with the corresponding pixels from the blurred version of the original image.

# My test will be performed below
Using my own code either at the end of this file or separately on my computer, I will just call the  ```AorO()``` or ```AorO2() functions in different ways and enjoy the outcomes :)